In [1]:
import pandas as pd
import feature_extractor as fe
from xgboost import XGBClassifier
from sklearn.metrics import *

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [2]:
data = pd.read_csv("updated_dataset.csv")
data

,length_url,length_hostname,ip,http_in_path,https_token,ratio_digits_url,ratio_digits_host,punycode,port,tld_in_path,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,status
0,37,19,0,0,1,0.000000,0.000000,0,0,0,...,0,0,1,0,45,-1,0,1,1,0
1,77,23,1,0,1,0.220779,0.000000,0,0,0,...,0,1,0,0,77,5767,0,0,1,1
2,126,50,1,0,0,0.150794,0.000000,0,0,0,...,0,1,0,0,14,4004,5828815,0,1,1
3,18,11,0,0,1,0.000000,0.000000,0,0,0,...,0,1,0,0,62,-1,107721,0,0,0
4,55,15,0,0,1,0.000000,0.000000,0,0,0,...,0,0,1,0,224,8175,8725,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,45,17,0,0,1,0.000000,0.000000,0,0,0,...,0,0,0,0,448,5396,3980,0,0,0
11426,84,18,0,0,1,0.023810,0.000000,0,0,1,...,0,1,0,0,211,6728,0,0,1,1
11427,105,16,1,0,0,0.142857,0.000000,0,0,0,...,0,0,0,0,2809,8515,8,0,1,0
11428,38,30,0,0,1,0.000000,0.000000,0,0,0,...,0,1,0,0,85,2836,2455493,0,0,0


In [6]:
def xgb_classifier(data=data, url=None):

    X = data.drop("status", axis=1)
    Y = data["status"]
    
    X_train, X_, Y_train, Y_ = train_test_split(X, Y, test_size=0.30, random_state=42)
    X_cv, X_test, Y_cv, Y_test = train_test_split(X_, Y_, test_size=0.50, random_state=42)
    del X_,Y_

    model = XGBClassifier(eta=0.1, max_depth=5, n_estimators=200, verbose=1, n_jobs=-1, num_parallel_tree=6, random_state=42)
    model.fit(X_train, Y_train, eval_set=[(X_cv, Y_cv)], early_stopping_rounds=20)
    
    if url:
        features = fe.extract_features(url)
        pred = model.predict(features)
        features.append(pred)
        print(pred)
        data.loc[len(df)] = features